In [ ]:
import sys
sys.path.append("..")
from src.load_data import load_dataframe_from_s3, save_dataframe_to_s3
from src.helpers import load_config_from_yaml
from src.data_prep import remove_gap
import numpy as np

In [ ]:
# Define config file path
conf_file_path = "../config/local.yaml"
# Load config
yaml_conf = load_config_from_yaml(file_path=conf_file_path)

# Get bucket name and data name from config.
bucket_name = yaml_conf.get("BUCKET_NAME")

In [ ]:
full_train_data = load_dataframe_from_s3(bucket_name, "prompt_engineering_300_original_reviewed.csv")

In [ ]:
# Count number of '<GAP>'s in the original data
full_train_data["REPORT"].str.count("<GAP>").sum()

In [ ]:
full_train_set_no_gap = full_train_data.apply(remove_gap, col="REPORT", axis=1)

In [ ]:
# Count number of '<GAP>'s in the new cleaned data
full_train_set_no_gap["REPORT"].str.count("<GAP>").sum()

In [ ]:
full_train_set_no_gap.dtypes

In [ ]:
#convert all value cols to string for ease of comparison in pipeline
score_columns = [f"ER_SCORE_{i+1}" for i in range(4)] + [f"PR_SCORE_{i+1}" for i in range(4)]
status_columns = [f"ER_STATUS_{i+1}" for i in range(4)] + [f"PR_STATUS_{i+1}" for i in range(4)] + [f"HER2_STATUS_{i+1}" for i in range(4)] + [f"HER2ISH_STATUS_{i+1}" for i in range(4)]

for col in score_columns:
    full_train_set_no_gap[col] = full_train_set_no_gap[col].apply(lambda x: str(int(float(x))) if not np.isnan(x) else None)

for col in status_columns:
    full_train_set_no_gap[col] = full_train_set_no_gap[col].apply(lambda x: str(x) if str(x) != 'nan' else None)

In [ ]:
full_train_set_no_gap.dtypes

In [ ]:
# description = "300 reports with '<GAP>' removed. Placeholders unaltered. Reviewed data."
# save_dataframe_to_s3(full_train_set_no_gap, bucket_name, "prompt_engineering_300_cleaned_reviewed.csv",description)